In [193]:
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
import tkinter as tk

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier, AdaBoostClassifier,VotingClassifier


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder

In [208]:
features,labels, model_l, log_reg_params = [], [], [], []
feature_train, label_train = pd.DataFrame(), pd.DataFrame()
def fileDialog():
        global filepath
        filepath = filedialog.askopenfilename(initialdir =  "/", title = "Select A File", 
                                                   filetype = (("CSV files","*.csv"),("EXCEL files","*.xlsx*")) )
#         filename = self.filepath.split('/')[-1]
#         print(self.filepath)
#         label = ttk.Label(labelFrame, text = "")
#         label.grid(column = 1, row = 2)
#         label.configure(text = filepath)
        parentlabelFrame = ttk.LabelFrame(text = "Select Features and Labels")
        parentlabelFrame.grid(row = 5, column = 30, padx = 20, pady = 20)
        
#         print(fileDialog.filepath)
        labelFrame = ttk.LabelFrame(parentlabelFrame, text = "Select Features")
        labelFrame.grid(row = 1, column = 0, padx = 20, pady = 20)
        
        data = pd.read_csv(filepath)
        columns = data.columns
#         features = []
        for i in range(len(columns)):
            temp = StringVar()
#             print(columns[i])
            ttk.Checkbutton(labelFrame, text = columns[i] , onvalue = columns[i], 
                                      offvalue = '', variable = temp).grid(row = i,column=0, sticky = tk.W)       
#                                       , command = lambda i=i: self.onClick(columns[i],features))

            global features
            features.append(temp)
#         for feature in features:
#             print(feature.get())
#         ttk.Button(labelFrame, text = "Go",command = split_data).grid(row = len(columns)+1, column =0)
        
        
        labelFrame = ttk.LabelFrame(parentlabelFrame, text = "Select Labels")
        labelFrame.grid(row = 1, column = 1, padx = 20, pady = 20)
        
        data = pd.read_csv(filepath)
        columns = data.columns
#         features = []
        for i in range(len(columns)):
            temp = StringVar()
#             print(columns[i])
            ttk.Checkbutton(labelFrame, text = columns[i] , onvalue = columns[i], 
                                       variable = temp).grid(row = i,column=0, sticky = tk.W)       
#                                       , command = lambda i=i: self.onClick(columns[i],features))

            global labels
            labels.append(temp)
#         for label in labels:
#             print(label.get())
            
        ttk.Button(parentlabelFrame, text = "Go",command = split_data).grid(row = len(columns)+1, column = 0)

def button(labelFrame):
        
        button = ttk.Button(labelFrame, text = "Browse A File",command = fileDialog)
        button.grid(row = 1, column = 1)
        
        
def split_data():
#             for feature in features:
#                 print(feature.get())
#             for label in labels:
#                 print(label.get())
                
            labelFrame = ttk.LabelFrame(text = "Enter Train - Test Split Percent")
            labelFrame.grid(row = 40, column = 30, padx  = 20, pady = 20)
        
            label = ttk.Label(labelFrame, text = "Train Split")
            label.grid(row = 0,column=0, sticky = tk.W)

            global train
            train = IntVar()
            
            nameEntered = ttk.Entry(labelFrame, width = 15, textvariable = train)
            nameEntered.grid(row = 0,column=1, sticky = tk.W)
            

            label = ttk.Label(labelFrame, text = "Test Split")
            label.grid(row = 1,column=0, sticky = tk.W)

            
            
            
            
            global test
            test = IntVar()
            nameEntered = ttk.Entry(labelFrame, width = 15, textvariable = test)
            nameEntered.grid(row = 1,column=1, sticky = tk.W)

            button = ttk.Button(labelFrame, text = "Compute", command = compute)
            button.grid(row = 3, column = 1)
#             temp.append(i)
#             temp[i] = StringVar()
#             feature = ttk.Checkbutton(self.labelFrame, text = columns[i] , 
#                                       state = "normal", onvalue = columns[i], 
#                                       offvalue = '', variable = temp)
# #                                       , command = lambda i=i: self.onClick(temp[i])
#             global features
#             features.append(temp)

        
        
        
#         self.button = ttk.Button(self.labelFrame, text = "Select Features",command = self.split_data)
#         self.button.grid(row = len(columns)+1, column = 0)
         
def compute():
    global features,labels, features_train, features_test, labels_train, labels_test

    lfeatures = []
    llabels = []
    print(test.get())
    data = pd.read_csv(filepath)
    for feature in features:
        if feature.get() != '':
            lfeatures.append(feature.get())
#         print(data[feature.get()])
    for label in labels:
        if label.get() != '':
            llabels.append(label.get())
#         print(data[label.get()])
#     print(lfeatures)
#     print(llabels)
    
    features = data[lfeatures]
#     print(features.head())
    labels = data[llabels]
#     print(labels.head())
#     print(data[lfeatures])
#     print(data[llabels])


    label_encoder = LabelEncoder()
    
    
    
    labels = label_encoder.fit_transform(labels)
    
#     print(labels)

    features_train, features_test, labels_train, labels_test = train_test_split(features , labels, test_size=test.get()/100, random_state=42)
    ml_models()


def model_tuner(features_train ,labels_train, features_test, labels_test,model,grid = None):
    
    if grid == None:
        clf = model
    else:
        clf = GridSearchCV(model,grid,n_jobs = -1)
    
    clf_fit = clf.fit(features_train, labels_train)
    
    if grid != None: 
        best_par = clf.best_params_
    
    labels_pred = clf.predict(features_test)
    p_pred = clf.predict_proba(features_test)
    cm = confusion_matrix(labels_test, labels_pred)
    accuracy = accuracy_score(labels_test, labels_pred)
    report = classification_report(labels_test,labels_pred)
    
    if grid != None: 
        print ('\nthe optimal parameters are: {}'.format(best_par))
    
    print ('\naccuracy on the test set is: {}'.format(accuracy))
    print ('\nconfusion matrix:\n\n{}'.format(cm))
    print ('\nclassification report:\n\n{}'.format(report))
    
    if grid != None:
        results_dict = {'best model':clf_fit,'best parameters':best_par, 'predicted values':labels_pred,
                    'predicted probabilities':p_pred,'confusion matrix':cm,'accuracy':accuracy,
                    'classification report':report}
    else:
        results_dict = {'best model':clf_fit, 'predicted values':labels_pred,'predicted probabilities':p_pred,
                        'confusion matrix':cm,'accuracy':accuracy,'classification report':report}
    return results_dict
    

    
def predict():
        
        global features_train
        global labels_train
        global features_test
        global labels_test
        global model_l
        
        checked_model = [] 
        for model in model_l:
            checked_model.append(model.get())
        
        random_state = 42
        
        if 'log_reg' in checked_model:
            log_mod = LogisticRegression(random_state = random_state)
            results_log = model_tuner(features_train,labels_train,features_test,labels_test,log_mod)
            print("Best Model\n",results_log['best model'],
                  "\nPredicted Values\n",results_log['predicted values'], 
                  "\nPredicted probabilities\n", results_log['predicted probabilities'],
                  "\nConfusion Matrix\n", results_log['confusion matrix'], 
                  "\nAccuracy\n", results_log['accuracy'], 
                  "\nClassification Report\n", results_log['classification report'])

        if 'DT' in checked_model:
            parameters_dt = {'max_features':list(range(1,4)), 'max_depth':list(np.arange(2,5))}
            dt_mod = DecisionTreeClassifier(random_state = random_state)
            
            results_dt = model_tuner(features_train, labels_train, features_test, labels_test,dt_mod,parameters_dt)
            print("\n--------------------------------------------------------------------------------------------\n",
                  "Best Model\n",results_dt['best model'],
                  "\nBest Parameters\n", results_dt['best parameters'],
                  "\nPredicted Values\n",results_dt['predicted values'], 
                  "\nPredicted probabilities\n", results_dt['predicted probabilities'],
                  "\nConfusion Matrix\n", results_dt['confusion matrix'], 
                  "\nAccuracy\n", results_dt['accuracy'], 
                  "\nClassification Report\n", results_dt['classification report'])
        
        if 'RF' in checked_model:
            parameters_rf = {'n_estimators':list(range(100,140,10)),'max_features':list(range(1,4))}
            rf = RandomForestClassifier(random_state = random_state)
            
            results_rf = model_tuner(features_train, labels_train, features_test, labels_test,rf,parameters_rf)
            
            print("\n--------------------------------------------------------------------------------------------\n",
                  "Best Model\n",results_rf['best model'],
                  "\nBest Parameters\n", results_rf['best parameters'],
                  "\nPredicted Values\n",results_rf['predicted values'], 
                  "\nPredicted probabilities\n", results_rf['predicted probabilities'],
                  "\nConfusion Matrix\n", results_rf['confusion matrix'], 
                  "\nAccuracy\n", results_rf['accuracy'], 
                  "\nClassification Report\n", results_rf['classification report'])
            
        

        
def ml_models():
            labelFrame = ttk.LabelFrame(text = "Select ML Models")
            labelFrame.grid(row = 60, column = 30, padx = 20, pady = 20)
            
            model_var = StringVar()
                              
            model1 = ttk.Checkbutton(labelFrame, text = "Logistic Regression", onvalue = 'log_reg', offvalue = None, variable = model_var).grid(row = 0,column=0, sticky = tk.W)
            
            model2 = ttk.Checkbutton(labelFrame, text = "Decision Tree", onvalue = 'DT', offvalue = None, variable = model_var).grid(row = 1,column =0, sticky = tk.W)
            
            model3 = ttk.Checkbutton(labelFrame, text = "Random Forest", onvalue = 'RF', offvalue = None, variable = model_var).grid(row = 2,column = 0, sticky = tk.W)
            
            global model_l
            model_l.append(model_var)
            
            button = ttk.Button(labelFrame, text = "Go", command = predict)
            button.grid(row = 3, column = 1)
            
            
 

    
root = Tk()
root.title("IRA AutoML Test")
root.minsize(400, 900)
rootHeight = root.winfo_height()
rootWidth = root.winfo_width()
root.wm_iconbitmap('favicon.ico')
labelFrame = ttk.LabelFrame(root, text = "Upload File")
labelFrame.grid(column = 30, row = 1, padx = 20, pady = 20)
button(labelFrame)
# split_data()


root.mainloop()